In [7]:
# Impor library yang dibutuhkan
import pandas as pd
import numpy as np

print("Library berhasil diimpor.")

Library berhasil diimpor.


In [8]:
# Membaca file CSV yang telah diunggah ke Google Colab
file_name = 'best_selling_books.csv'

try:
    df = pd.read_csv(file_name)
    print(f"'{file_name}' berhasil dimuat.")
except FileNotFoundError:
    print(f"ERROR: File '{file_name}' tidak ditemukan. Pastikan Anda sudah mengunggahnya.")
    # Membuat DataFrame contoh jika file tidak ada
    data = {'Book': ['Book A', 'Book B', 'Book C'],
            'Author(s)': ['Author 1', 'Author 2', 'Author 3'],
            'Original language': ['English', 'English', 'French'],
            'First published': [1999, 2005, '1988'],
            'Approximate sales in millions': ['120', '85-95', '70'],
            'Genre': ['Fantasy', 'Fantasy', 'Adventure']}
    df = pd.DataFrame(data)
    print("Membuat DataFrame contoh untuk melanjutkan demonstrasi.")

ERROR: File 'best_selling_books.csv' tidak ditemukan. Pastikan Anda sudah mengunggahnya.
Membuat DataFrame contoh untuk melanjutkan demonstrasi.


In [9]:
# Menampilkan 5 baris pertama dari dataset
print("--- 5 Baris Pertama Dataset ---")
display(df.head())

# Menampilkan informasi ringkas tentang DataFrame
print("\n--- Informasi Awal Dataset ---")
df.info()

# Memeriksa jumlah nilai kosong (missing values) di setiap kolom
print("\n--- Jumlah Nilai Kosong per Kolom ---")
print(df.isnull().sum())

--- 5 Baris Pertama Dataset ---


,Book,Author(s),Original language,First published,Approximate sales in millions,Genre
0,Book A,Author 1,English,1999,120,Fantasy
1,Book B,Author 2,English,2005,85-95,Fantasy
2,Book C,Author 3,French,1988,70,Adventure



--- Informasi Awal Dataset ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Book                           3 non-null      object
 1   Author(s)                      3 non-null      object
 2   Original language              3 non-null      object
 3   First published                3 non-null      object
 4   Approximate sales in millions  3 non-null      object
 5   Genre                          3 non-null      object
dtypes: object(6)
memory usage: 276.0+ bytes

--- Jumlah Nilai Kosong per Kolom ---
Book                             0
Author(s)                        0
Original language                0
First published                  0
Approximate sales in millions    0
Genre                            0
dtype: int64


In [10]:
# Salin dataframe untuk menjaga data asli
df_processed = df.copy()

# --- Langkah 1: Membersihkan nama kolom dari spasi yang tidak perlu ---
df_processed.columns = df_processed.columns.str.strip()

# --- Langkah 2: Mengisi nilai kosong di kolom 'Genre' ---
if df_processed['Genre'].isnull().any():
    df_processed['Genre'].fillna('Unknown', inplace=True)
    print("Nilai kosong di kolom 'Genre' telah diisi dengan 'Unknown'.")

# --- Langkah 3: Membersihkan dan mengubah kolom 'Approximate sales in millions' ---
def clean_sales(sales):
    if isinstance(sales, str):
        sales = sales.strip()
        if '-' in sales:
            try:
                low, high = map(float, sales.split('-'))
                return (low + high) / 2
            except ValueError:
                return np.nan # Kembalikan NaN jika konversi gagal
        try:
            return float(sales)
        except ValueError:
            return np.nan # Kembalikan NaN jika konversi gagal
    return sales

df_processed['avg_sales_millions'] = df_processed['Approximate sales in millions'].apply(clean_sales)

# --- Langkah 4: Mengubah 'First published' menjadi tipe data numerik (tahun) ---
df_processed['publication_year'] = pd.to_numeric(df_processed['First published'], errors='coerce')

# Mengisi nilai tahun yang kosong (jika ada) dengan median
if df_processed['publication_year'].isnull().any():
    median_year = df_processed['publication_year'].median()
    df_processed['publication_year'].fillna(median_year, inplace=True)
    print(f"Nilai tahun publikasi yang kosong diisi dengan median: {int(median_year)}")

# Mengubah tipe data tahun menjadi integer
df_processed['publication_year'] = df_processed['publication_year'].astype(int)

# --- Langkah 5: Menghapus kolom asli yang sudah tidak relevan ---
df_processed.drop(['Approximate sales in millions', 'First published'], axis=1, inplace=True)

# --- Langkah 6: Menangani duplikat ---
rows_before = len(df_processed)
df_processed.drop_duplicates(inplace=True)
rows_after = len(df_processed)
if rows_before > rows_after:
    print(f"{rows_before - rows_after} baris data duplikat telah dihapus.")

print("\nPra-pemrosesan data selesai.")


Pra-pemrosesan data selesai.


In [11]:
# Menampilkan 5 baris pertama dari dataset yang sudah diproses
print("--- 5 Baris Pertama Dataset yang Sudah Diproses ---")
display(df_processed.head())

# Menampilkan informasi ringkas tentang DataFrame yang sudah diproses
print("\n--- Informasi Dataset Setelah Pra-pemrosesan ---")
df_processed.info()

# Memeriksa kembali nilai kosong untuk memastikan data sudah bersih
print("\n--- Jumlah Nilai Kosong Setelah Pembersihan ---")
print(df_processed.isnull().sum())

--- 5 Baris Pertama Dataset yang Sudah Diproses ---


,Book,Author(s),Original language,Genre,avg_sales_millions,publication_year
0,Book A,Author 1,English,Fantasy,120.0,1999
1,Book B,Author 2,English,Fantasy,90.0,2005
2,Book C,Author 3,French,Adventure,70.0,1988



--- Informasi Dataset Setelah Pra-pemrosesan ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book                3 non-null      object 
 1   Author(s)           3 non-null      object 
 2   Original language   3 non-null      object 
 3   Genre               3 non-null      object 
 4   avg_sales_millions  3 non-null      float64
 5   publication_year    3 non-null      int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 276.0+ bytes

--- Jumlah Nilai Kosong Setelah Pembersihan ---
Book                  0
Author(s)             0
Original language     0
Genre                 0
avg_sales_millions    0
publication_year      0
dtype: int64


In [12]:
# Menyimpan dataframe yang sudah bersih ke file CSV baru
output_filename = 'best_selling_books_processed.csv'
df_processed.to_csv(output_filename, index=False)

print(f"Data yang sudah bersih telah berhasil disimpan ke file '{output_filename}'")

Data yang sudah bersih telah berhasil disimpan ke file 'best_selling_books_processed.csv'


# New Section

# New Section